In [1]:
import sagemaker
import boto3
import pandas as pd 
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\venka\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'panchabucket'
print('using bucket name',bucket)

using bucket name panchabucket


In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
df.shape

(2000, 21)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [7]:
features = list(df.columns)

In [8]:
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [9]:
target = features.pop(-1)

In [10]:
x = df[features]
y = df[target]

In [11]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [12]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [13]:
y

0       1
1       2
2       2
3       2
4       1
       ..
1995    0
1996    2
1997    3
1998    0
1999    3
Name: price_range, Length: 2000, dtype: int64

In [14]:
y.value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [15]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [16]:
X_train.shape

(1700, 20)

In [17]:
X_test.shape

(300, 20)

In [18]:
train = X_train
train[target] = y_train

test = X_test
test[target] = y_test

In [19]:
train.shape

(1700, 21)

In [20]:
test.shape

(300, 21)

In [21]:
train.to_csv("train_data.csv")
test.to_csv("test_data.csv")

In [21]:
sk_prefix = 'sagemaker/mobile_price_classfication/sklearncontainer'
train_path = sess.upload_data(
    path="train_data.csv",bucket=bucket,key_prefix=sk_prefix
)
test_path = sess.upload_data(
    path="test_data.csv",bucket=bucket,key_prefix=sk_prefix
)
print(train_path)
print(test_path)

s3://panchabucket/sagemaker/mobile_price_classfication/sklearncontainer/train_data.csv
s3://panchabucket/sagemaker/mobile_price_classfication/sklearncontainer/test_data.csv


In [22]:
%%writefile script.py
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np 
import pandas as pd 

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir,"model.joblib"))
    return clf

if __name__ == "__main__":
    print("[INFO] Extracting arguments..")
    parser = argparse.ArgumentParser()

    # Hyperparameters sent by the client
    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--random_state",type=int,default=0)

    # Data, model and output directories
    parser.add_argument("--model_dir",type=str,default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test",type=str,default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file",type=str,default="train_data.csv")
    parser.add_argument("--test-file",type=str,default="test_data.csv")

    args,_ = parser.parse_known_args()

    print("SKlearn Version: ",sklearn.__version__)
    print("JobLIb version:", joblib.__version__)

    print("[INFO] Reading data..")
    print()

    train_df = pd.read_csv(os.path.join(args.train,args.train_file))
    test_df = pd.read_csv(os.path.join(args.test,args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets..")
    print()

    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print("Column order...:")
    print(features)
    print()

    print("Label column is:",label)
    print()

    print("SHAPE of TRAINING DATA:")
    print(X_train.shape)
    print(y_train.shape)
    print()

    print("SHAPE of TESTING DATA:")
    print(X_test.shape)
    print(y_test.shape)

    print("Training Random Forest model..")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,verbose=0,n_jobs=-1)
    model.fit(X_train,y_train)
    print()

    model_path = os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at:",model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("--METRICS RESULTS FOR TESTING DATA--")
    print()
    print("Total Rows are:",X_test.shape[0])
    print("[INFO] Testing model accuracy is: ",test_acc)
    print(test_rep)


Writing script.py


In [23]:
from sagemaker.sklearn.estimator import SKLearn
FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::008971668552:role/service-role/AmazonSageMaker-ExecutionRole-20240924T145373",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0
    },
    use_spot_instances = True,
    max_wait=7200,
    max_run=3600
)

In [24]:
sklearn_estimator.fit({"train":train_path,"test":test_path},wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-09-30-15-25-22-672
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'arn:aws:iam::008971668552:user/sagemakerex' at 'roleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$